# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [14]:
r = requests.get(url)
r.status_code
#r.content

200

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [127]:
from bs4 import BeautifulSoup as bs

In [128]:
sopa = bs(r.text,'lxml')
links=[a for a in sopa.find_all('a', href=True)]

In [129]:
#Marcar como false todo lo que no empieze con <a href=
for i in range(len(links)):
    links[i]=str(links[i])
    if links[i].find('<a href="/wiki/') !=0:
        links[i]=False
#Remover de la lista los False
for i in range(links.count(False)):
    links.remove(False) 
#Quitar etiquetas y href y title con su texto
for i in range(len(links)):
    links[i]=links[i].replace('<a href="/wiki','https://en.wikipedia.org/wiki')
    fin=links[i].find('"')
    links[i]=links[i][:fin]
    


In [130]:
print('Links extraidos:',len(links))
links = set(links)
links = list(links)
print('Links unicos:',len(links))

Links extraidos: 208
Links unicos: 160


In [133]:
links.sort()
links

['https://en.wikipedia.org/wiki/Academic_journal',
 'https://en.wikipedia.org/wiki/Academic_publishing',
 'https://en.wikipedia.org/wiki/Academy',
 'https://en.wikipedia.org/wiki/American_Statistical_Association',
 'https://en.wikipedia.org/wiki/Analytics',
 'https://en.wikipedia.org/wiki/Anomaly_detection',
 'https://en.wikipedia.org/wiki/Applied_science',
 'https://en.wikipedia.org/wiki/Artificial_neural_network',
 'https://en.wikipedia.org/wiki/Association_rule_learning',
 'https://en.wikipedia.org/wiki/Autoencoder',
 'https://en.wikipedia.org/wiki/Automated_machine_learning',
 'https://en.wikipedia.org/wiki/BIRCH',
 'https://en.wikipedia.org/wiki/Basic_research',
 'https://en.wikipedia.org/wiki/Bayesian_network',
 'https://en.wikipedia.org/wiki/Big_data',
 'https://en.wikipedia.org/wiki/Boosting_(machine_learning)',
 'https://en.wikipedia.org/wiki/Bootstrap_aggregating',
 'https://en.wikipedia.org/wiki/Category:All_articles_with_unsourced_statements',
 'https://en.wikipedia.org/wik

In [100]:
type(links.count(False))
len(links)

329

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [ ]:
domain = 'http://wikipedia.org'

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [ ]:
import os

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [ ]:
from slugify import slugify

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [ ]:
import multiprocessing